In [1]:
import pandas as pd

In [2]:
emails = pd.read_csv('emails.csv')
# email_subset = emails[:10000]
email_subset = emails.sample(frac=0.02, random_state=1)

print(email_subset.shape)
print(email_subset.head())

(10348, 2)
                              file  \
186822  jones-t/all_documents/634.   
308790  mann-k/all_documents/5690.   
82383         dasovich-j/sent/423.   
227299          kaminski-v/var/63.   
301824     mann-k/_sent_mail/3208.   

                                                  message  
186822  Message-ID: <17820178.1075846925335.JavaMail.e...  
308790  Message-ID: <29110382.1075845717882.JavaMail.e...  
82383   Message-ID: <6812040.1075843194135.JavaMail.ev...  
227299  Message-ID: <21547648.1075856642126.JavaMail.e...  
301824  Message-ID: <12684200.1075846107179.JavaMail.e...  


In [3]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [4]:
def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from')
    }


In [5]:
def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [6]:
email_df = pd.DataFrame(parse_into_emails(email_subset.message))
print(email_df.head())

                                                body  \
0  It would be nice if you could be at my dinner,...   
1  Absolutely.Good point!  Can Peter start to dra...   
2  My apologies.  My schedule melted down after w...   
3  Vince,UK VAR breached the limit last week.UK t...   
4  Any problems/comments?AM ---------------------...   

                                                  to  \
0                           alicia.goodrow@enron.com   
1                          Kay Mann/Corp/Enron@ENRON   
2                        christine.piesco@oracle.com   
3  Richard Lewis/LON/ECT@ECT, James New/LON/ECT@E...   
4  Don Hammond/PDX/ECT@ECT, Jody Blackburn/PDX/EC...   

                               from_  
0               tana.jones@enron.com  
1  Sheila Tweed@ECT on 05/15/2001 06  
2            jeff.dasovich@enron.com  
3        tanya.tamarchenko@enron.com  
4                 kay.mann@enron.com  


In [7]:
import re
import numpy as np

In [8]:
import gensim

In [9]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [10]:
# spacy for lemmatization
import spacy

In [11]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [12]:
#  >>> import nltk
#  >>> nltk.download('stopwords')

In [13]:
# prep NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""


In [14]:
print(email_df.iloc[2]['body']) # displays info below

My apologies.  My schedule melted down after we talked on Monday.  Here'swhere folks came out.  There's some concern about size.  We're supposed to beno larger than 3, but we lobbied Aceves and he apparently Ok'd our"oversized" group.  The other folks in the group--who talked to himoriginally--are pretty sure that five will violate the rules.  Folks wonderedif there were other groups that are smaller than ours that you could hook upwith.  Sorry about that---it's a wrinkle that I didn't think about when wespoke.  If it gets real ugly trying to find a smaller group, let me know.Fortunately there's not another team case due for two weeks.Best,Jeff


In [15]:
# Convert email body to list
data = email_df.body.values.tolist()

In [16]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [17]:
data_words = list(sent_to_words(data))

In [18]:
print(data_words[3])

['vince', 'uk', 'var', 'breached', 'the', 'limit', 'last', 'week', 'uk', 'traders', 'asked', 'us', 'to', 'review', 'the', 'correlations', 'across', 'uk', 'gas', 'and', 'power', 'aswell', 'as', 'the', 'correlations', 'across', 'efa', 'slots', 'we', 'did', 'part', 'of', 'the', 'work', 'last', 'week', 'now', 'we', 'll', 'update', 'the', 'correlations', 'based', 'on', 'historical', 'prices', 'tanya', 'richard', 'lewisleppard', 'lon', 'ect', 'ect', 'rudy', 'dautel', 'hou', 'ect', 'ect', 'kirstee', 'hewitt', 'lon', 'ect', 'ect', 'naveen', 'andrews', 'corp', 'enron', 'enron', 'david', 'port', 'market', 'risk', 'corp', 'enron', 'enron', 'tedmurphy', 'hou', 'ect', 'ect', 'simon', 'hastings', 'lon', 'ect', 'ect', 'paul', 'arcy', 'lon', 'ect', 'ect', 'amirghodsian', 'lon', 'ect', 'ectthanks', 'tanya', 'these', 'are', 'interesting', 'results', 'am', 'on', 'vacation', 'next', 'week', 'sohere', 'are', 'my', 'current', 'thoughts', 'am', 'contactable', 'on', 'my', 'mobile', 'if', 'necessary', 'gas', '

In [19]:
from gensim.models.phrases import Phrases, Phraser

In [20]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [21]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [22]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])

['image_image_image', 'getting', 'together', 'for', 'the', 'holidays', 'is', 'something', 'we', 'all', 'enjoy', 'whether', 'it', 'gathering', 'with', 'old', 'friends', 'from', 'out', 'of', 'town', 'or', 'hanging', 'out', 'with', 'the', 'usual', 'gang', 'every', 'gathering', 'this', 'time', 'of', 'year', 'seems', 'little', 'more', 'special', 'we', 'at', 'miller_brewing', 'wish', 'you', 'many', 'happy', 'celebrations', 'this', 'season', 'and', 'thank', 'you', 'for', 'enjoying', 'those', 'occasions', 'responsibly', 'happy_holidays', 'image', 'this', 'mail', 'is', 'not', 'sent', 'unsolicited', 'you', 'subscribed', 'to', 'receive', 'information', 'from', 'miller_brewing', 'at', 'miller', 'web_site', 'or', 'event', 'must', 'be', 'or', 'older', 'to', 'visit', 'our', 'web_site', 'miller_brewing', 'co', 'milwaukee', 'wi', 'miller_brewing', 'company', 'milwaukee', 'wi', 'privacy_statement', 'image_image_image_image', 'this', 'message', 'was', 'sent', 'by', 'miller_brewing', 'company', 'using', '

In [23]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [25]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [26]:
#python -m spacy download en

In [27]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

In [28]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [29]:
print(data_lemmatized[200])

['image_image', 'image', 'get', 'together', 'holiday', 'something', 'enjoy', 'gather', 'old', 'friend', 'town', 'hang', 'usual', 'gang', 'gathering', 'time', 'year', 'seem', 'little', 'special', 'miller_brew', 'wish', 'many', 'happy', 'celebration', 'season', 'thank', 'enjoy', 'occasion', 'responsibly', 'happy_holiday', 'image', 'mail', 'send', 'unsolicited', 'subscribed', 'receive', 'information', 'miller_brew', 'miller', 'web_site', 'event', 'must', 'old', 'visit', 'web_site', 'miller_brew', 'co', 'milwaukee', 'wi', 'miller_brew', 'company', 'milwaukee', 'wi', 'privacy', 'statement', 'image_image', 'image_image', 'message', 'send', 'miller_brew', 'company', 'use', 'responsy', 'interact', 'tm', 'click', 'prefer', 'receive', 'future', 'mail', 'miller_brew', 'company', 'click', 'view', 'permission', 'marketing', 'policy', 'image']


In [30]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [37]:
#!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [32]:
#!unzip mallet-2.0.8.zip

In [33]:
mallet_path = './mallet-2.0.8/bin/mallet'

In [34]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [36]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [ ]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

In [ ]:
doc_lda = lda_model[corpus]

In [ ]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [ ]:
pyLDAvis.display(vis)

In [ ]:
# now using mallet

In [ ]:
mallet_path = 'mallet-2.0.8/bin/mallet'

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
print(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
# # Visualize the topics with mallet model
# pyLDAvis.enable_notebook(sort=True)
# vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
# pyLDAvis.display(vis)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# run
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)


In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[4]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)



In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

In [ ]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
# Show
df_dominant_topic.head(10)

In [ ]:
df_dominant_topic.Keywords.iloc[1]

In [ ]:
df_dominant_topic.Text.iloc[1]

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

In [ ]:
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

In [ ]:
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

In [ ]:
# Show
sent_topics_sorteddf_mallet